In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
data = pd.read_csv('/root/autodl-tmp/dataset/UserBehavior.csv',header=None)
data = data.sample(frac=0.1)

In [3]:
data.columns = ['userId','itemId','catId','userBehavior','logTs']

In [4]:
data['label_fav'] = data['userBehavior'].map(lambda x : 1 if x == 'fav' else 0)
data['label_buy'] = data['userBehavior'].map(lambda x : 1 if x == 'buy' else 0)
data['label_cart'] = data['userBehavior'].map(lambda x : 1 if x == 'cart' else 0)

In [5]:
# def object2int(x):
#     if x == 'pv':
#         x = 0
#     elif(x == 'fav'):
#         x = 1
#     elif(x == 'buy'):
#         x = 2 
#     elif(x == 'cart'):
#         x = 3  
#     return x
# data['userBehavior'] = data.userBehavior.apply(object2int)

# Shared_bottem


1. input

In [27]:
inputs = {
    # 'userId':tf.keras.layers.Input(name='userId', shape=(), dtype='int32'),
    'itemId':tf.keras.layers.Input(name='itemId', shape=(), dtype='int32'),
    # 'catId' :tf.keras.layers.Input(name='catId', shape=(), dtype='int32'),

}    # 'userBehavior' :tf.keras.layers.Input(name='userBehavior', shape=(), dtype='int32'),

In [28]:
data.itemId.max()

5163067

In [29]:
item_features = tf.keras.layers.Embedding(5163071, 10 )(inputs['itemId'])

In [30]:
shared_bottom = tf.keras.layers.Dense(32,activation='relu',name='shared_bottom')(item_features)
tower_fav = tf.keras.layers.Dense(16,activation='relu',name='tower_fav')(shared_bottom)
tower_buy = tf.keras.layers.Dense(16,activation='relu',name='tower_buy')(shared_bottom)
tower_cart = tf.keras.layers.Dense(16,activation='relu',name='tower_cart')(shared_bottom)


In [31]:
output_fav = tf.keras.layers.Dense(1,activation='sigmoid',name='output_fav')(tower_fav)
output_buy = tf.keras.layers.Dense(1,activation='sigmoid',name='output_buy')(tower_buy)
output_cart = tf.keras.layers.Dense(1,activation='sigmoid',name='output_cart')(tower_cart)

In [32]:
model = tf.keras.Model(inputs,outputs=[output_fav,output_buy,output_cart] )

In [35]:
x_train = data.itemId.values

In [14]:
label_fav = data.pop('label_fav')
label_buy = data.pop('label_buy')
label_cart = data.pop('label_cart')

In [41]:
model.compile(optimizer='sgd',
              loss={'output_fav': 'binary_crossentropy', 
              'output_buy': 'binary_crossentropy',
              'output_cart': 'binary_crossentropy'},
              loss_weights={'output_fav': 0.5, 'output_buy': 0.2,'output_cart':0.3})

In [43]:
model.fit( x_train,
          {'output_fav': label_fav , 'output_buy': label_buy,'output_cart':label_cart},
          epochs=1, batch_size=16)